<a href="https://colab.research.google.com/github/josepmatias/nos_reten-ao/blob/main/elegiveis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import pandas as pd
import matplotlib.pyplot as plt
import dask.dataframe as dd

In [41]:
!pip install dask[dataframe] pyxlsb

In [42]:
#especificar o nome do ficheiro
resumo_r_nome="Resumo_Reativa_Setembro23.xlsb"
resumo_r1_nome="Resumo_Reativa_novembro23.xlsb"

In [44]:
#importar ficheiros

from google.colab import files
uploaded = files.upload()
uploaded2=files.upload()

Saving Resumo_Reativa_Setembro23.xlsb to Resumo_Reativa_Setembro23.xlsb


Saving Resumo_Reativa_novembro23.xlsb to Resumo_Reativa_novembro23.xlsb


In [45]:
#criar resumo mês n
resumo_r=pd.read_excel(resumo_r_nome, sheet_name="Auto", engine="pyxlsb")

In [46]:
#criar resumo mês n-3

resumo_r1=pd.read_excel(resumo_r1_nome, sheet_name="Auto", engine="pyxlsb")

In [47]:
#renomear colunas da bd consoante reactiva ou proactiva

resumo_n=resumo_r.copy()
resumo_n1=resumo_r1.copy()

def renomear_cols (df):
  lista=df.columns.tolist()
  lista2=[]
  for i in lista:
    if i.find("RRR")!=-1:
      lista2.append(i.replace("RRR", "").strip())
    elif i.find("RPR")!=-1:
      lista2.append(i.replace("RPR", "").strip())
    else:
      lista2.append(i)
  return lista2

resumo_n.columns=renomear_cols(resumo_n)
resumo_n1.columns=renomear_cols(resumo_n1)

In [48]:
#extrair linhas para avaliar se são elegíveis para cb - event type name="SERVICE REQUEST" e "ACTIVAÇÃO DE SERVIÇO"

def event_sr (df, ciclo, evento=["SERVICE REQUEST", "ACTIVAÇÃO DE SERVIÇO"]):
  if ciclo=="RRR":
    lista_incentivo=["RRR_CHARGEBACK NOS", 'RRR_PREVISÃO DE CHARGEBACK NOS']
    for i in evento:
      if i=="SERVICE REQUEST":
        df_sr=df.loc[(~df["Incentive Type Name"].isin(lista_incentivo)) & (df["Event Type Name"]==i), :]
      elif i=="ACTIVAÇÃO DE SERVIÇO":
        df_ativacao=df.loc[(~df["Incentive Type Name"].isin(lista_incentivo)) & (df["Event Type Name"]==i), :]
  else:
    lista_incentivo=["RPR_CHARGEBACK NOS", 'RPR_PREVISÃO DE CHARGEBACK NOS']
    for i in evento:
      if i=="SERVICE REQUEST":
        df_sr=df.loc[(~df["Incentive Type Name"].isin(lista_incentivo)) & (df["Event Type Name"]==i), :]
      elif i=="ACTIVAÇÃO DE SERVIÇO":
        df_ativacao=df.loc[(~df["Incentive Type Name"].isin(lista_incentivo)) & (df["Event Type Name"]==i), :]


  return df_sr, df_ativacao


df_sr, df_ativacao=event_sr(resumo_n, "RRR")



In [57]:
#extrair linhas para avaliar se são elegíveis para cb - Chargeback Reason=Activação Negativa + portabilidades

def event_cb (df, df1, col=["Chargeback Reason", "Event Type Name"], str="Activação Negativa", str2=["EVENTO GENÉRICO DE SERVIÇO", "PORTABILIDADE DE SERVIÇO"]):
  for i in col:
    if i=="Chargeback Reason":
      df_cb=df1.loc[df1[i]==str, :]
    else:
      df_portabilidade=df.loc[(df[i].isin(str2)) & (df["Incentive Type Name"].str.find("PORTABILIDADE")!=-1), :]
  return df_cb, df_portabilidade

df_cb, df_portabilidade=event_cb(resumo_n, resumo_n1)

In [60]:
df_cb.shape

(34, 101)

In [ ]:
#add different dataframes to the same excel file

#excel_file_path = 'multi_sheet_excel.xlsx'
#with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    #df1.to_excel(writer, sheet_name='Sheet1', index=False)
    #df2.to_excel(writer, sheet_name='Sheet2', index=False)
    #df3.to_excel(writer, sheet_name='Sheet3', index=False)